# Decorators
- Solos e importables como módulos.

In [31]:
## Standard Libs
from typing import Union, Optional, Tuple, Dict, Any
import warnings
import random
import re

# Third-Party Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rich.console import Console
from rich.text import Text
from rich.panel import Panel

# Local Libs
from jm_datascience import jm_pandas as jm_pd
from jm_datascience import jm_pdaccessor

## DECORATOR -
- Es función debería estar en jm_utils porque puede modificar cualquier numerom
- Podría usarse como decorator pero como decorator cómo la aplico a series o df?

In [32]:
# 1. definir función formatear número
def fmt_number(value, width=8, decimals=2, miles=',') -> str:
    if not isinstance(width, int) or width <= 0:
        raise ValueError(f"Width must be a positive integer. Not '{width}'")
    
    if not isinstance(decimals, int) or decimals < 0:
        raise ValueError(f"Decimals must be a non-negative integer. Not '{decimals}")
    
    if miles not in [',', '_', None]:
        raise ValueError(f"Miles must be either ',', '_', or None. Not '{miles}")
    
    try:
        num = float(value)                                  # Convert to float if possible
        if miles:
            return f"{num:>{width}{miles}.{decimals}f}"     # Ancho fijo, x decimales, alineado a la derecha
        else:
            return f"{num:>{width}.{decimals}f}"
        
    except (ValueError, TypeError) as e:
        # return str(value).rjust(width)                            # Alinea también strings, para mantener la grilla
        return f"[ERROR] Al tratar de 'float({value})' - {e}"   
      
# 1b. decorater
def fmt_numbers(width=8, decimals=2, miles=','):
    def _decorador(func):
        def wrapper(*args, **kwargs):
            resultado = func(*args, **kwargs)
            return fmt_number(resultado, width, decimals, miles)
        return wrapper
    return _decorador

In [33]:
# 2. A sample funct.
# print(eval(input('Enter: ')))
@ fmt_numbers()
def maths_ops(operation: str):
    return eval(operation)

print(maths_ops('15000 - 500.9'))
# res = maths_ops('15000 - 500.9')
# print(f"{res = } | {type(res) = }")

14,499.10


In [34]:
def print_decorator(width=20, decimals=1, miles='_'):
    """Decorador específico para funciones que imprimen resultados."""
    def _decorador(func):
        def wrapper(*args, **kwargs):
            nuevos_args = []
            for arg in args:
                if isinstance(arg, (int, float, list, tuple, dict, str)):
                    nuevos_args.append(fmt_number(arg, width, decimals, miles))
                else:
                    nuevos_args.append(arg)
            return func(*nuevos_args, **kwargs)
        return wrapper
    return _decorador

In [35]:
# 3. si quisiera decorar la función print tengo que
@ print_decorator()
def jmprt(*args):
    print(*args)

print(55300.1234)
jmprt(55300.1234)

55300.1234
            55_300.1


## Format numbers 
- Funciones para formatear núemros ya se que están solo o en collectios

In [ ]:
def fmt_num(value, decimals=2, miles=',') -> str:
    try:
        num = float(value)                                  # Convert to float if possible
        if miles:
            return f"{num:>{miles}.{decimals}f}"     
        else:
            return f"{num:>.{decimals}f}"
        
    except (ValueError, TypeError) as e:                          #
        return f"[ERROR] Al tratar de 'float({value})' - {e}"   
    
    
def fmt_num_in_str(text, decimals=2, miles=','):
    """Search and format numbers within a string."""
    def _reeplace(match):
        number = float(match.group(0))
        return fmt_num(number, decimals, miles)
    
    pattern = r"[-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?"
    return re.sub(pattern, _reeplace, text)


def fmt_nums(value, decimals=2, miles=','):
    try:
        value = float(value)
    except:
        if isinstance(value, list):
            return [fmt_nums(element, decimals, miles) for element in value]
        if isinstance(value, list):
            return tuple([fmt_nums(element, decimals, miles) for element in value])
        elif isinstance(value, dict):
            return {k: fmt_nums(v, decimals, miles) for k, v in value.items()}
        elif isinstance(value, str):
            return fmt_num_in_str(value, decimals, miles)
        
    else:
        return fmt_num(value, decimals, miles)

In [71]:
print(fmt_num(98765, decimals=0))
print(fmt_num_in_str('Precio: 150000.1934', decimals=1, miles='_'))
fmt_nums('Precio: 150000.1934', decimals=1, miles='_')

98,765
Precio: 150_000.2


'Precio: 150_000.2'

In [68]:
random.random() * 10000

4420.348363931854

In [69]:
lst = [random.random() * 10_000 for i in range(7)]
lst

[4070.003043625502,
 9340.103219453278,
 274.19755395085343,
 2288.5968204450623,
 3939.840061237535,
 620.9252281405575,
 3756.240880359113]

In [70]:
print(fmt_nums(898783425.8993))

898,783,425.90
